In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import RobustScaler
import os

# ======================================================
# CẤU HÌNH (SỬA TẠI ĐÂY)
# ======================================================
TEST_FILE = 'du_lieu_test_no_label.csv'      # File dữ liệu mới
OUTPUT_FILE = 'ket_qua_du_doan_single.csv'   # Tên file kết quả
BEST_MODEL_PATH = 'saved_models/model_fold_4.keras' # <-- THAY SỐ 3 BẰNG SỐ FOLD TỐT NHẤT CỦA BẠN

# Tọa độ AOI (Giữ nguyên)
AOI_EYES_X = 512; AOI_EYES_Y = 350
AOI_MOUTH_X = 512; AOI_MOUTH_Y = 550

# ======================================================
# 1. HÀM XỬ LÝ DỮ LIỆU
# ======================================================
def preprocess_new_data(file_path):
    print(f" Đang xử lý file: {file_path}")
    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(" Lỗi: Không tìm thấy file.")
        return None, None

    # Tính AOI
    df['DIST_EYES'] = np.sqrt((df['FIX_X'] - AOI_EYES_X)**2 + (df['FIX_Y'] - AOI_EYES_Y)**2)
    df['DIST_MOUTH'] = np.sqrt((df['FIX_X'] - AOI_MOUTH_X)**2 + (df['FIX_Y'] - AOI_MOUTH_Y)**2)

    # Hàm Pivot
    def pivot_feature(df, value_col):
        # Tạo index tùy thuộc vào cột có sẵn
        keys = ['numeric_id', 'image_id']
        if 'label' in df.columns: keys.append('label')
            
        df_pivoted = df.pivot_table(index=keys, columns='FIX_INDEX', values=value_col).reset_index()
        step_cols = [i for i in range(1, 21)]
        df_pivoted[step_cols] = df_pivoted[step_cols].bfill(axis=1).ffill(axis=1)
        return df_pivoted, df_pivoted[step_cols].values

    # Pivot & Baseline Correction
    df_meta, pupil_values = pivot_feature(df, 'FIX_PUPIL')
    pupil_baseline = pupil_values[:, 0].reshape(-1, 1)
    pupil_values = pupil_values - pupil_baseline

    _, dur_values = pivot_feature(df, 'FIX_DURATION')
    _, deye_values = pivot_feature(df, 'DIST_EYES') 
    _, dmou_values = pivot_feature(df, 'DIST_MOUTH')

    # Stacking
    X_new = np.stack([pupil_values, dur_values, deye_values, dmou_values], axis=2)
    ids_new = df_meta['numeric_id'].values
    
    return ids_new, X_new

# ======================================================
# 2. CHẠY DỰ ĐOÁN VỚI 1 MODEL
# ======================================================
def run_single_prediction():
    # 1. Chuẩn bị dữ liệu
    ids_test, X_test = preprocess_new_data(TEST_FILE)
    if X_test is None: return

    # 2. Chuẩn hóa
    print(" Đang chuẩn hóa dữ liệu...")
    for i in range(4):
        scaler = RobustScaler()
        X_test[:, :, i] = scaler.fit_transform(X_test[:, :, i])

    # 3. Load Model & Dự đoán
    print(f" Đang tải mô hình tốt nhất: {BEST_MODEL_PATH}")
    if os.path.exists(BEST_MODEL_PATH):
        model = tf.keras.models.load_model(BEST_MODEL_PATH)
        
        # Dự đoán
        raw_predictions = model.predict(X_test, verbose=1).flatten()
        
        # 4. Tổng hợp kết quả
        results_df = pd.DataFrame({
            'numeric_id': ids_test,
            'prob': raw_predictions
        })

        # Tính trung bình cho từng người
        final_report = results_df.groupby('numeric_id').agg(
            avg_prob=('prob', 'mean')
        ).reset_index()

        # Ra quyết định
        final_report['Prediction'] = (final_report['avg_prob'] > 0.5).astype(int)
        final_report['Label_Name'] = final_report['Prediction'].map({0: 'Bình thường', 1: 'ASD'})

        # Lưu file
        final_report.to_csv(OUTPUT_FILE, index=False)
        print(f"\n XONG! Kết quả đã lưu tại: {OUTPUT_FILE}")
        print(final_report.head())
        
    else:
        print(f" Lỗi: Không tìm thấy file model tại {BEST_MODEL_PATH}")

# Chạy
if __name__ == "__main__":
    run_single_prediction()

 Đang xử lý file: du_lieu_test_no_label.csv
 Đang chuẩn hóa dữ liệu...
 Đang tải mô hình tốt nhất: saved_models/model_fold_4.keras
150/150 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step

 XONG! Kết quả đã lưu tại: ket_qua_du_doan_single.csv
   numeric_id  avg_prob  Prediction   Label_Name
0           0  0.394830           0  Bình thường
1           1  0.867830           1          ASD
2           2  0.636813           1          ASD
3           3  0.373816           0  Bình thường
4           4  0.505564           1          ASD
